In [1]:
# For target_1 "at_risk_event".
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.svm import OneClassSVM
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix


data = pd.read_csv('D:/dissertation/session 1/feature.csv')

# Define the features that will be used by the models.
features = ['is_non_working_hour', 'is_weekend', 'adjusted_non_working_hour_risk_normalized',
            'adjusted_weekend_risk_normalized', 'first_non_working_hour', 'first_weekend', 
            'time_to_working_hour_normalized', 'spend_normalized', 'spend_diff_flag']

# Define specific parameters for One-Class SVM that will be used for each cluster.
params = {
    0: {'nu': 0.03, 'kernel': 'rbf', 'gamma': 'auto'},
    1: {'nu': 0.03, 'kernel': 'rbf', 'gamma': 'auto'},
    2: {'nu': 0.04, 'kernel': 'rbf', 'gamma': 'auto'}
}

results = []
test_results = []

for cluster in data['cluster_label'].unique():
    # Filter data for each cluster.
    cluster_data = data[data['cluster_label'] == cluster]

    # Select features and target variable.
    X = cluster_data[features]
    y = cluster_data['at_risk_event'].apply(lambda x: 1 if x else 0)  # 确保标签正确转换

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Split data into training and test sets.
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

    # Initialize One-Class SVM with parameters specific to the cluster.
    oc_svm = OneClassSVM(**params[cluster])

    # Train the model on the training data.
    oc_svm.fit(X_train)

    # Predict on the test set.
    y_pred_test = oc_svm.predict(X_test)
    y_pred_test = [0 if x == 1 else 1 for x in y_pred_test]

    # Append the test results for evaluation.
    test_results.append(pd.DataFrame({
        'true_label': y_test,
        'predicted_label': y_pred_test
    }))

# Combine all test results into a single DataFrame for evaluation.
final_test_results = pd.concat(test_results)

# Calculate and print performance metrics.
precision = precision_score(final_test_results['true_label'], final_test_results['predicted_label'])
recall = recall_score(final_test_results['true_label'], final_test_results['predicted_label'])
f1 = f1_score(final_test_results['true_label'], final_test_results['predicted_label'])

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

# Print the confusion matrix.
conf_matrix = confusion_matrix(final_test_results['true_label'], final_test_results['predicted_label'])
print("Confusion Matrix:")
print(conf_matrix)


Precision: 0.2168
Recall: 0.2804
F1 Score: 0.2445
Confusion Matrix:
[[19980   542]
 [  385   150]]


In [3]:
# For target_2 "at_risk_event".
import pandas as pd
from sklearn.svm import OneClassSVM

data = pd.read_csv('D:/dissertation/session 1/feature.csv')

# Define model parameters for One-Class SVM for each cluster.
params = {
    0: {'kernel': 'rbf', 'nu': 0.01, 'gamma': 'auto'},
    1: {'kernel': 'rbf', 'nu': 0.01, 'gamma': 'auto'},
    2: {'kernel': 'rbf', 'nu': 0.02, 'gamma': 'auto'}
}

# Define the list of features that will be used in the anomaly detection model.
features = ['is_non_working_hour', 'is_weekend', 'prob_non_working_hours_normalized', 'prob_weekend_normalized',
            'first_and_second_non_working_hour', 'first_and_second_weekend', 
            'spend_normalized', 'spend_diff', 'spend_diff_flag', 'first_two_high_spend']

test_results = []

# Split the dataset by cluster label and train One-Class SVM for each cluster.
for cluster_label in data['cluster_label'].unique():
    cluster_data = data[data['cluster_label'] == cluster_label]

    # Train the One-Class SVM model using the defined parameters for the cluster.
    ocsvm = OneClassSVM(**params[cluster_label])
    ocsvm.fit(cluster_data[features])
    
    # Predict anomalies and calculate decision function scores.
    cluster_data['scores'] = ocsvm.decision_function(cluster_data[features])
    cluster_data['outlier_label'] = ocsvm.predict(cluster_data[features])

    # Convert the outlier labels from -1 and 1 to 1 and 0.
    cluster_data['outlier_label'] = (cluster_data['outlier_label'] == -1).astype(int)

    # Save the prediction results for further analysis.
    test_results.append(cluster_data)

# Combine the results from all clusters into a single DataFrame.
final_test_data = pd.concat(test_results)

final_test_data['date'] = pd.to_datetime(final_test_data['date'])
final_test_data.sort_values(by=['individual_id', 'date'], inplace=True)
final_test_data['window_id'] = pd.NA
final_test_data['processed_flag'] = False

# Detect and mark anomaly windows.
window_id = 0
for index, row in final_test_data.iterrows():
    if row['outlier_label'] == 1 and not row['processed_flag']:
        current_datetime = row['date']
        end_datetime = current_datetime + pd.Timedelta(days=4)
        
        # Select records within a 5-day window.
        mask = (final_test_data['individual_id'] == row['individual_id']) & \
               (final_test_data['date'] >= current_datetime) & \
               (final_test_data['date'] <= end_datetime)
        
        # Start marking from the current record to ensure the window starts from the current anomaly.
        current_record_index = final_test_data[(final_test_data['individual_id'] == row['individual_id']) & \
                                               (final_test_data['date'] == current_datetime)].index
        start_index = current_record_index[current_record_index >= index][0]
        
        mask = (mask) & (final_test_data.index >= start_index)
        
        # Assign a window ID to all records within the window and mark them as processed.
        final_test_data.loc[mask, 'window_id'] = window_id
        final_test_data.loc[mask, 'processed_flag'] = True
        window_id += 1

final_test_data.drop(columns=['processed_flag'], inplace=True)


C:\Users\DELL\AppData\Local\Temp\ipykernel_9948\1681223035.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cluster_data['scores'] = ocsvm.decision_function(cluster_data[features])
C:\Users\DELL\AppData\Local\Temp\ipykernel_9948\1681223035.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cluster_data['outlier_label'] = ocsvm.predict(cluster_data[features])
C:\Users\DELL\AppData\Local\Temp\ipykernel_9948\1681223035.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from 

In [4]:
def second_target_eval_func(df,
                            second_target_pred_col_name="second_target_pred",
                            at_risk_behaviour_window_col_name="at_risk_behaviour_window"):
    """
    Evaluate the predictions for the second target, at_risk_behaviour_window.

    Args:
        df (pandas.DataFrame): the dataframe with your predictions and the target window.
        second_target_pred_col_name (string): the name of the column with your predictions.
        at_risk_behaviour_window_col_name (string): the name of the column with the target window.

    Returns:
        pandas.DataFrame: the original dataframe with 2 additional columns:
            target_window_found: has value True if window found, False if window wasnt found, NaN if event isnt in a window;
            correct_predictions: has value True if prediction was in a window, False if outside, NaN if no prediction made.
        float: precision.
        float: recall.
    """

    df_copy = df.copy()

    # Identify the windows found.
    windows_found = df_copy.groupby(at_risk_behaviour_window_col_name).agg({second_target_pred_col_name:"max"})[second_target_pred_col_name]
    windows_missed = ~windows_found

    # Join the windows found with the df.
    df_copy = pd.merge(df_copy, pd.DataFrame(windows_found), left_on=at_risk_behaviour_window_col_name, right_index=True, how="left", suffixes=("", "_found"))
    df_copy = df_copy.rename(columns={second_target_pred_col_name + "_found":"target_window_found"})

    # Identify the events where a prediction was correctly or incorrectly made.
    correct_predictions = df_copy[df_copy[second_target_pred_col_name]][at_risk_behaviour_window_col_name].notna()
    df_copy["correct_predictions"] = correct_predictions
    num_predictions = sum(df_copy[second_target_pred_col_name])
    missed_predictions = (correct_predictions == False).sum()

    # Calculate metrics.
    """
    TP is the number of windows correctly identified.
    FP is the number of predictions made incorrectly.
    Additional predictions made for a window after the first do not affect the precision or recall (ie the FP wont change).
    """
    TP = windows_found.sum()
    FP = missed_predictions
    support = len(windows_found)
    precision = TP/(TP + FP)
    recall = TP/support

    return df_copy, precision, recall

In [5]:
def check_window_id(row):
    if not pd.isna(row['window_id']):
        return True
    else:
        return False

final_test_data["my_pred_column"] = final_test_data.apply(lambda row: check_window_id(row), axis=1)

In [6]:
df_with_pred_evaluations, precision, recall = second_target_eval_func(df=final_test_data,
                                                                      second_target_pred_col_name="my_pred_column")

print("precision of OCSVM:", precision, "recall of OCSVM:", recall)

precision of OCSVM: 0.03764225269915378 recall of OCSVM: 0.1419141914191419
